<a href="https://colab.research.google.com/github/semkud/NLP/blob/main/HW2/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для начала загрузим все библиотеки и укажем наши тексты и списки с эталонными вариантами разметки.

In [ ]:
!pip install natasha

In [ ]:
!pip install slovnet


In [ ]:
!pip install navec

In [ ]:
!pip install pymorphy2
!pip install pymystem3 --upgrade
% pip install pymystem3

In [130]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [129]:
import navec
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

text = 'Вы когда-нибудь были на Таити? Не были? Ну и правильно, к чему бы вам там бывать, если только не для того чтобы окунуться с головой в озеро Лумпнуть, потревожить его гладь, рубашку скинуть и нырять, а потом жарить шашлыки в гостинной. Перелет до Таити будет 3-часовым, а на борту вам предложат минеральную, газированную, натриево карбонатную и магниевую воду, чтобы вы, как восьмидесятипроцентный водный человек восполнили свой баланс и были готовы загорать, изжариваться под лучами знойного таитского солнца. Если у вас стекло в кармане, не бойтесь проходить таможню, там сидят абсолютно слепые люди, у одного такого слепого даже стекло по штанине мороженное, а он не заметил.'

Сложные места это:
1) Имена собственные - Таити, Лумпнуть
2) слово гладь, за которым следует рубашка
3) отадъективы типа мороженное, гостинная, слепой
4) стекло как существительное и как глагол

По очереди анализируем текст - Наташей, майстемом, пайморфи

In [74]:
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
display(doc.tokens[9].pos)
#doc.sents[0].morph.print()
natasha_version = []
for token in doc.tokens:
  if token.pos != 'PUNCT':
    natasha_version.append([token.text.lower(), token.pos])

'PART'

In [75]:
len(natasha_version)

105

In [76]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem
import pymystem3
import json
from pymystem3 import Mystem
m = Mystem()
a = m.analyze(text)

--2020-10-13 15:45:34--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.242, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm907.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-13 15:45:35--  http://cache-mskm907.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm907.cdn.yandex.net (cache-mskm907.cdn.yandex.net)... 5.45.220.17, 2a02:6b8:0:2002::18
Connecting to cache-mskm907.cdn.yandex.net (cache-mskm907.cdn.yandex.net)|5.45.220.17|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.4’

mystem-3.0-linux3.1 100%[===================>] 

In [77]:
mystem_version = []
len(mystem_version)
for word in a:
    if 'analysis' in word:
      if len(word['analysis']) > 0:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        r = word['text']
        mystem_version.append([r.lower(), pos])

In [78]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

pymorphy_version = []
tt = word_tokenize(text)
for t in tt:
  if 'PNCT' not in str(morph.parse(t)[0].tag)[0:4]:
    pymorphy_version.append([morph.parse(t)[0].word, str(morph.parse(t)[0].tag)[0:4]])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Создаем функцию Унифай - приводящая все теги к одним

In [87]:
PRON = ['PRON', 'SPRO', 'NPRO', 'APRO', 'DET']
ADVB = ['ADV', 'ADVPRO', 'ADVB']
PREP = ['ADP', 'PR', 'PREP']
NOUN = ['PROPN', 'S', 'NOUN']
PART = ['PART', 'PRCL']
VERB = ['AUX', 'V', 'VERB', 'INFN', 'PRTF']
CONJ = ['CCONJ', 'CONJ', 'SCONJ']
ADJ = ['ADJF', 'ADJ', 'A', 'ADJS', 'ADJF']
def unify(c):
  for a in range(0, len(c), 1):
    if c[a][1] in PRON:
      c[a][1] = 'PRON'
    elif c[a][1] in ADVB:
      c[a][1] = 'ADVB'
    elif c[a][1] in PREP:
      c[a][1] = 'PREP'
    elif c[a][1] in NOUN:
      c[a][1] = 'NOUN'
    elif c[a][1] in PART:
      c[a][1] = 'PART'
    elif c[a][1] in VERB:
      c[a][1] = 'VERB'
    elif c[a][1] in CONJ:
      c[a][1] = 'CONJ'
    elif c[a][1] in ADJ:
      c[a][1] = 'ADJ'
  return(c)



In [88]:
nv = unify(natasha_version)
nm = unify(mystem_version)
np = unify(pymorphy_version)

А это наш Золотой стандарт ручной разметки

In [89]:
gold = [['вы', 'PRON'], ['когда-нибудь', 'ADVB'], ['были', 'VERB'], ['на', 'PREP'], ['таити', 'NOUN'],
        ['не', 'PART'], ['были', 'VERB'], ['ну', 'PART'], ['и', 'CONJ'], ['правильно', 'ADVB'], ['к', 'PREP'],
        ['чему', 'PRON'], ['бы', 'PART'], ['вам', 'PRON'], ['там', 'ADVB'], ['бывать', 'VERB'], ['если', 'CONJ'],
        ['только', 'ADVB'], ['не', 'PART'], ['для', 'PREP'], ['того', 'PRON'], ['чтобы', 'CONJ'], ['окунуться', 'VERB'],
        ['с', 'PREP'], ['головой', 'NOUN'], ['в', 'PREP'], ['озеро', 'NOUN'], ['лумпнуть', 'NOUN'], ['потревожить', 'VERB'],
        ['его', 'PRON'], ['гладь', 'NOUN'], ['рубашку', 'NOUN'], ['скинуть', 'VERB'], ['и', 'CONJ'], ['нырять', 'VERB'],
        ['а', 'CONJ'], ['потом', 'ADVB'], ['жарить', 'VERB'], ['шашлыки', 'NOUN'], ['в', 'PREP'], ['гостинной', 'NOUN'],
        ['перелёт', 'NOUN'], ['до', 'PREP'], ['таити', 'NOUN'], ['будет', 'VERB'], ['3-часовым', 'ADJ'], ['а', 'CONJ'],
        ['на', 'PREP'], ['борту', 'NOUN'], ['вам', 'PRON'], ['предложат', 'VERB'], ['минеральную', 'ADJ'],
        ['газированную', 'VERB'], ['натриево', 'ADJ'], ['карбонатную', 'ADJ'], ['и', 'CONJ'], ['магниевую', 'ADJ'],
        ['воду', 'NOUN'], ['чтобы', 'CONJ'], ['вы', 'PRON'], ['как', 'CONJ'], ['восьмидесятипроцентный', 'ADJ'],
        ['водный', 'ADJ'], ['человек', 'NOUN'], ['восполнили', 'VERB'], ['свой', 'PRON'], ['баланс', 'NOUN'], ['и', 'CONJ'],
        ['были', 'VERB'], ['готовы', 'ADJ'], ['загорать', 'VERB'], ['изжариваться', 'VERB'], ['под', 'PREP'], ['лучами', 'NOUN'],
        ['знойного', 'ADJ'], ['таитского', 'PRON'], ['солнца', 'NOUN'], ['если', 'CONJ'], ['у', 'PREP'], ['вас', 'PRON'],
        ['стекло', 'NOUN'], ['в', 'PREP'], ['кармане', 'NOUN'], ['не', 'PART'], ['бойтесь', 'VERB'], ['проходить', 'VERB'],
        ['таможню', 'NOUN'], ['там', 'ADVB'], ['сидят', 'VERB'], ['абсолютно', 'ADVB'], ['слепые', 'ADJ'], ['люди', 'NOUN'],
        ['у', 'PREP'], ['одного', 'PRON'], ['такого', 'PRON'], ['слепого', 'NOUN'], ['даже', 'PART'], ['стекло', 'VERB'],
        ['по', 'PREP'], ['штанине', 'NOUN'], ['мороженное', 'NOUN'], ['а', 'CONJ'], ['он', 'PRON'], ['не', 'PART'],
        ['заметил', 'VERB']]

Функция, которой мы протестируем разные разметки

In [90]:
def test(a, b):
  count = 0
  for q in range(0, len(a),1):
    if a[q][1] == b[q][1]:
      count = count + 1
  return(count*100/len(a))

In [91]:
print('Точность Наташи - ', test(nv, gold), '\n', 'Точность майстем - ', test(nm, gold), '\n','Точность пайморфи - ', test(np, gold), )

Точность Наташи -  91.42857142857143 
 Точность майстем -  92.38095238095238 
 Точность пайморфи -  87.61904761904762


ЧАСТЬ С АНГЛИЙСКИМ

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [99]:
text2 = 'It usually snows heavily in Babylon in the morning. All Mesapotamia is covered with snow. Many people get corneal burns while looking out into that whitish desert. Many desert burn fires whistling their favorite song and can be seen from afar. They order drinks and their screams freeze the soul. There are other guys here, George. This is not Germany, this is not Afghanistan. George, your soldiers will be torn apart here. These are 250 thousand elite soldiers of the Babylonian Kingdom! They will smash everything! They will cover the entire desert in one hour! They will blow up all your destroyers, all your journalists, diplomats. George, you are a cowboy. You stop, damn it, you finish, you hide the cartridges away in the warehouse. And forget about your dad.'

В начале много конверсий - snow, snows, burns, burn, drinks, whistle, desert, freeze

In [ ]:
import spacy

# Загружаем весь пайплайн для английского
nlp = spacy.load("en_core_web_sm")

# Обрабатываем текст
doc = nlp(text2)
spacy_version = []
# Выведем токены, леммы и теги
for i, s in enumerate(doc.sents):
    print("\n-- Sentence %d --" % i)
    for t in s:
        spacy_version.append([t.text.lower(), t.pos_])

In [110]:
print(spacy_version)

[['it', 'PRON'], ['usually', 'ADV'], ['snows', 'VERB'], ['heavily', 'ADV'], ['in', 'ADP'], ['babylon', 'PROPN'], ['in', 'ADP'], ['the', 'DET'], ['morning', 'NOUN'], ['.', 'PUNCT'], ['all', 'DET'], ['mesapotamia', 'PROPN'], ['is', 'AUX'], ['covered', 'VERB'], ['with', 'ADP'], ['snow', 'NOUN'], ['.', 'PUNCT'], ['many', 'ADJ'], ['people', 'NOUN'], ['get', 'VERB'], ['corneal', 'ADJ'], ['burns', 'NOUN'], ['while', 'SCONJ'], ['looking', 'VERB'], ['out', 'ADP'], ['into', 'ADP'], ['that', 'DET'], ['whitish', 'ADJ'], ['desert', 'NOUN'], ['.', 'PUNCT'], ['many', 'ADJ'], ['desert', 'NOUN'], ['burn', 'VERB'], ['fires', 'NOUN'], ['whistling', 'VERB'], ['their', 'DET'], ['favorite', 'ADJ'], ['song', 'NOUN'], ['and', 'CCONJ'], ['can', 'VERB'], ['be', 'AUX'], ['seen', 'VERB'], ['from', 'ADP'], ['afar', 'ADV'], ['.', 'PUNCT'], ['they', 'PRON'], ['order', 'VERB'], ['drinks', 'NOUN'], ['and', 'CCONJ'], ['their', 'DET'], ['screams', 'NOUN'], ['freeze', 'VERB'], ['the', 'DET'], ['soul', 'NOUN'], ['.', 'PUN

Как мы видим, Спайси раскусил все конверсии кроме немного кривоватого предложения про "многие дезертируют, жгут костры..."


Кстати, сразу заведем золотой стандарт:

In [111]:
gold2 = [['it', 'PRON'], ['usually', 'ADV'], ['snows', 'VERB'], ['heavily', 'ADV'], ['in', 'ADP'], 
         ['babylon', 'PROPN'], ['in', 'ADP'], ['the', 'DET'], ['morning', 'NOUN'], ['.', 'PUNCT'], 
         ['all', 'DET'], ['mesapotamia', 'PROPN'], ['is', 'AUX'], ['covered', 'VERB'], ['with', 'ADP'], 
         ['snow', 'NOUN'], ['.', 'PUNCT'], ['many', 'ADJ'], ['people', 'NOUN'], ['get', 'VERB'], ['corneal', 'ADJ'], 
         ['burns', 'NOUN'], ['while', 'SCONJ'], ['looking', 'VERB'], ['out', 'ADP'], ['into', 'ADP'], ['that', 'DET'], 
         ['whitish', 'ADJ'], ['desert', 'NOUN'], ['.', 'PUNCT'], ['many', 'ADJ'], ['desert', 'VERB'], ['burn', 'VERB'], 
         ['fires', 'NOUN'], ['whistling', 'VERB'], ['their', 'DET'], ['favorite', 'ADJ'], ['song', 'NOUN'], ['and', 'CCONJ'], 
         ['can', 'VERB'], ['be', 'AUX'], ['seen', 'VERB'], ['from', 'ADP'], ['afar', 'ADV'], ['.', 'PUNCT'], ['they', 'PRON'], 
         ['order', 'VERB'], ['drinks', 'NOUN'], ['and', 'CCONJ'], ['their', 'DET'], ['screams', 'NOUN'], ['freeze', 'VERB'], 
         ['the', 'DET'], ['soul', 'NOUN'], ['.', 'PUNCT'], ['there', 'PRON'], ['are', 'AUX'], ['other', 'ADJ'], ['guys', 'NOUN'], 
         ['here', 'ADV'], [',', 'PUNCT'], ['george', 'PROPN'], ['.', 'PUNCT'], ['this', 'DET'], ['is', 'AUX'], ['not', 'PART'], 
         ['germany', 'PROPN'], [',', 'PUNCT'], ['this', 'DET'], ['is', 'AUX'], ['not', 'PART'], ['afghanistan', 'PROPN'], 
         ['.', 'PUNCT'], ['george', 'PROPN'], [',', 'PUNCT'], ['your', 'DET'], ['soldiers', 'NOUN'], ['will', 'VERB'], 
         ['be', 'AUX'], ['torn', 'VERB'], ['apart', 'ADV'], ['here', 'ADV'], ['.', 'PUNCT'], ['these', 'DET'], ['are', 'AUX'], 
         ['250', 'NUM'], ['thousand', 'NUM'], ['elite', 'ADJ'], ['soldiers', 'NOUN'], ['of', 'ADP'], ['the', 'DET'], 
         ['babylonian', 'PROPN'], ['kingdom', 'PROPN'], ['!', 'PUNCT'], ['they', 'PRON'], ['will', 'VERB'], ['smash', 'VERB'], 
         ['everything', 'PRON'], ['!', 'PUNCT'], ['they', 'PRON'], ['will', 'VERB'], ['cover', 'VERB'], ['the', 'DET'], 
         ['entire', 'ADJ'], ['desert', 'NOUN'], ['in', 'ADP'], ['one', 'NUM'], ['hour', 'NOUN'], ['!', 'PUNCT'], ['they', 'PRON'], 
         ['will', 'VERB'], ['blow', 'VERB'], ['up', 'ADP'], ['all', 'DET'], ['your', 'DET'], ['destroyers', 'NOUN'], [',', 'PUNCT'],
         ['all', 'DET'], ['your', 'DET'], ['journalists', 'NOUN'], [',', 'PUNCT'], ['diplomats', 'NOUN'], ['.', 'PUNCT'], 
         ['george', 'PROPN'], [',', 'PUNCT'], ['you', 'PRON'], ['are', 'AUX'], ['a', 'DET'], ['cowboy', 'NOUN'], ['.', 'PUNCT'], 
         ['you', 'PRON'], ['stop', 'VERB'], [',', 'PUNCT'], ['damn', 'VERB'], ['it', 'PRON'], [',', 'PUNCT'], ['you', 'PRON'], 
         ['finish', 'VERB'], [',', 'PUNCT'], ['you', 'PRON'], ['hide', 'VERB'], ['the', 'DET'], ['cartridges', 'NOUN'], 
         ['away', 'ADV'], ['in', 'ADP'], ['the', 'DET'], ['warehouse', 'NOUN'], ['.', 'PUNCT'], ['and', 'CCONJ'], 
         ['forget', 'VERB'], ['about', 'ADP'], ['your', 'DET'], ['dad', 'NOUN'], ['.', 'PUNCT']]

In [ ]:
! pip install flair

In [112]:
from flair.data import Sentence
from flair.models import SequenceTagger
import re

# загружаем модель теггинга
tagger = SequenceTagger.load('pos')



sentences = re.split('\. ', text2)
for s in sentences:
  sentence = Sentence(s)
  tagger.predict(sentence)
  print('####')
  print(sentence.to_tagged_string())

2020-10-13 21:03:21,184 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt
####
It <PRP> usually <RB> snows <VBZ> heavily <RB> in <IN> Babylon <NNP> in <IN> the <DT> morning <NN>
####
All <DT> Mesapotamia <NNP> is <VBZ> covered <VBN> with <IN> snow <NN>
####
Many <JJ> people <NNS> get <VBP> corneal <JJ> burns <NNS> while <IN> looking <VBG> out <RB> into <IN> that <DT> whitish <JJ> desert <NN>
####
Many <JJ> desert <NN> burn <NN> fires <NNS> whistling <VBG> their <PRP$> favorite <JJ> song <NN> and <CC> can <MD> be <VB> seen <VBN> from <IN> afar <RB>
####
They <PRP> order <VBP> drinks <NNS> and <CC> their <PRP$> screams <NNS> freeze <VBP> the <DT> soul <NN>
####
There <EX> are <VBP> other <JJ> guys <NNS> here <RB> , <,> George <NNP>
####
This <DT> is <VBZ> not <RB> Germany <NNP> , <,> this <DT> is <VBZ> not <RB> Afghanistan <NNP>
####
George <NNP> , <,> your <PRP$> soldiers <NNS> will <MD> be <VB> torn <VBN> apart <RB> here <RB>
####
These <DT> are <VBP> 250 <CD> thousand <CD> eli

Этот тоже не справился с many desert

In [118]:
import nltk
nltk.download('averaged_perceptron_tagger')


texxt = word_tokenize(text2)
nltk_version = nltk.pos_tag(texxt)
print(nltk_version)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[('It', 'PRP'), ('usually', 'RB'), ('snows', 'VBZ'), ('heavily', 'RB'), ('in', 'IN'), ('Babylon', 'NNP'), ('in', 'IN'), ('the', 'DT'), ('morning', 'NN'), ('.', '.'), ('All', 'DT'), ('Mesapotamia', 'NNP'), ('is', 'VBZ'), ('covered', 'VBN'), ('with', 'IN'), ('snow', 'JJ'), ('.', '.'), ('Many', 'JJ'), ('people', 'NNS'), ('get', 'VBP'), ('corneal', 'JJ'), ('burns', 'NNS'), ('while', 'IN'), ('looking', 'VBG'), ('out', 'RP'), ('into', 'IN'), ('that', 'DT'), ('whitish', 'JJ'), ('desert', 'NN'), ('.', '.'), ('Many', 'JJ'), ('desert', 'JJ'), ('burn', 'NN'), ('fires', 'NNS'), ('whistling', 'VBG'), ('their', 'PRP$'), ('favorite', 'NN'), ('song', 'NN'), ('and', 'CC'), ('can', 'MD'), ('be', 'VB'), ('seen', 'VBN'), ('from', 'IN'), ('afar', 'NN'), ('.', '.'), ('They', 'PRP'), ('order', 'NN'), ('drinks', 'NNS'), ('an

Как мы видим, разметка НЛТК полностью совпадает с разметкой flair

В последней части напишем с помощью Наташи функцию, находящую в тексте все пары слов типа "не + прилагательное" - понятно зачем, "наречие + прилагательное" - чтобы искать конструкции со словами самый абсолютно очень и т.д., "наречие + глагол" - тоже добавляет эмоциональной окраски, типа очень нравится, надолго завораживает - экспрессивные обороты, которые могут помочь в тональном словаре

In [139]:

def function1(text) :
  list_adv_adj = []
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for i in range(0, len(doc.tokens), 1):
    if doc.tokens[i].pos == 'ADV' and doc.tokens[i+1].pos == 'ADJ':
      list_adv_adj.append(''.join([doc.tokens[i].text.lower(), ' ', doc.tokens[i+1].text.lower()]))
  return(list_adv_adj)

def function2(text) :
  list_ne_adj = []
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for i in range(0, len(doc.tokens), 1):
    if doc.tokens[i].text.lower() == 'не' and doc.tokens[i+1].pos == 'ADJ':
      list_adv_adj.append(''.join([doc.tokens[i].text.lower(), ' ', doc.tokens[i+1].text.lower()]))
  return(list_ne_adj)

def function3(text) :
  list_adv_verb = []
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for i in range(0, len(doc.tokens), 1):
    if doc.tokens[i].pos == 'ADV' and doc.tokens[i+1].pos == 'VERB':
      list_adv_adj.append(''.join([doc.tokens[i].text.lower(), ' ', doc.tokens[i+1].text.lower()]))
  return(list_adv_verb)

Проверим на нашем тексте из первой части программы:

In [148]:
print(function1(text))
print(function2(text))
print(function3(text))

['абсолютно слепые']
[]
[]
